In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [113]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    #r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
r = requests.get("https://www.amazon.com/Microban-Disinfectant-Sanitizing-Spray-Citrus/product-reviews/B085S97MWM/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=1" , headers=headers)
content = r.content
soup = BeautifulSoup(content, 'html.parser')

In [114]:
print(soup.title.text)

Amazon.com: Customer reviews: Microban 24 Hour Disinfectant Sanitizing Spray, Citrus Scent, 2 Count, 15 fl oz Each


In [119]:
no_pages = 16

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    #r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    r = requests.get("https://www.amazon.com/Microban-Disinfectant-Sanitizing-Spray-Citrus/product-reviews/B085S97MWM/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&reviewerType=all_reviews&pageNumber=" + str(pageNo) , headers=headers)
    content = r.content
    soup = BeautifulSoup(content, 'html.parser')
    #print(soup)
    

    alls = []
    for d in soup.findAll('div', attrs={'data-hook':'review'}):
        #print(d)
        #name = d.find('span', attrs={'class':'a-profile-name'})
        
        title = d.find('a', attrs={'data-hook':'review-title'})
        rating = d.find('i', attrs={'data-hook':'review-star-rating'})
        body = d.find('span', attrs={'data-hook':'review-body'})
        date = d.find('span', attrs={'data-hook':'review-date'})
        product_title = soup.title.text.replace('Amazon.com: Customer reviews:', '')
        

        all1=[]

        if title is not None:
            #print(n[0]['alt'])
            all1.append(title.text)
        else:
            all1.append("unknown-product")

#         if rating is not None:
#             #print(author.text)
#             all1.append(rating.text)
#         elif author is None:
#             author = d.find('span', attrs={'class':'a-size-small a-color-base'})
#             if author is not None:
#                 all1.append(author.text)
#             else:    
#                 all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('0')

        if body is not None:
            #print(price.text)
            all1.append(body.text)
        else:
            all1.append('unknown')     

        if date is not None:
            #print(price.text)
            all1.append(date.text)
        else:
            all1.append('0')
            
        if product_title is not None:
            #print(rating.text)
            all1.append(product_title)
#         else:
#             all1.append('unknown_product')  
            
        alls.append(all1)    
    return alls

In [120]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Title', 'rating','body','date','product_title'])
df.to_excel('amazon_products3.xlsx', index=False, encoding='utf-8')

In [121]:
df = pd.read_excel("amazon_products3.xlsx")

In [122]:
df.head()

,Title,rating,body,date,product_title
0,\nmicroban kills viruses\n,5.0 out of 5 stars,\n\n Microban most definitely kills viruses a...,"Reviewed in the United States on August 15, 2020",Microban 24 Hour Disinfectant Sanitizing Spra...
1,\nP n g tried to copy Lysol and clorox but con...,3.0 out of 5 stars,\n\n Do not confuse this with absolute vivid ...,"Reviewed in the United States on June 23, 2020",Microban 24 Hour Disinfectant Sanitizing Spra...
2,\nHuge bottle.\n,5.0 out of 5 stars,\n\n I have never seen this spray before and ...,"Reviewed in the United States on September 2, ...",Microban 24 Hour Disinfectant Sanitizing Spra...
3,\nVery bad experience\n,1.0 out of 5 stars,\n\n Received without lid. No scent at all. M...,"Reviewed in the United States on November 14, ...",Microban 24 Hour Disinfectant Sanitizing Spra...
4,\nIt's ok but a little pricey\n,4.0 out of 5 stars,\n\n I like it but it only kills bacteria and...,"Reviewed in the United States on June 30, 2020",Microban 24 Hour Disinfectant Sanitizing Spra...


In [123]:
df.shape

(156, 5)

In [124]:
df.tail()

,Title,rating,body,date,product_title
151,\nGood\n,5.0 out of 5 stars,\n\n Good\n\n,"Reviewed in the United States on November 5, 2020",Microban 24 Hour Disinfectant Sanitizing Spra...
152,\nTerrible product - spray stops working\n,1.0 out of 5 stars,\n\n I've bought 3 of these sprays during the...,"Reviewed in the United States on October 8, 2020",Microban 24 Hour Disinfectant Sanitizing Spra...
153,\nThis specific Microban does not kill COVID\n,3.0 out of 5 stars,\n\n This particular Microban product DOES NO...,"Reviewed in the United States on November 5, 2020",Microban 24 Hour Disinfectant Sanitizing Spra...
154,\nA wet mess that takes forever to dry\n,3.0 out of 5 stars,\n\n Good product but my only negative comme...,"Reviewed in the United States on December 12, ...",Microban 24 Hour Disinfectant Sanitizing Spra...
155,\nKills da germs!!!\n,5.0 out of 5 stars,\n\n Disinfects without strong chemical smell...,"Reviewed in the United States on December 3, 2020",Microban 24 Hour Disinfectant Sanitizing Spra...
